# Value at Risk 101: A simple way to monitor market risk with Atoti

Financial institutions all have to find a balance between profit and risk. The more risk taken the higher the profit can be, but then if we want to avoid collapses such as that of Lehman Brothers in 2008, risk has to be controlled.  

There are several kinds of risk:
- Shortfall of a counterparty also know as credit risk: This is the risk that a borrower cannot pay its credit
- Market risk: This is the risk that certain assets could lose their value. For example one might invest in wine bottle in the hope that they gain value with age while they might not.

Market risk is widely monitored in finance. Institutions have large portfolios with a lot of assets, and forecasting the value of each asset is simply impossible as COVID-19 kindly reminded it to us recently. The key is then to assess what are the (statistical) chances that the value of certain assets remain in a certain enveloppe and what are potential losses. This is where the value at risk – or VaR – comes into action.

In this notebook we will show you how we can compute and aggregate pretty easily this non-linear indicator with Atoti, and then perform simulations around it.

## Importing the necessary libraries

In [1]:
import pandas as pd
import atoti
import os.path
import wget
# You will need wget to load the data this notebook uses.
# If you do not already have it installed: conda install python-wget

Welcome to Atoti 0.3.1!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


## Data Loading
#### Initializing Atoti

In [2]:
from atoti.config import create_config
#tell atoti to load the database containing the UI dashboards
config = create_config(metadata_db="./metadata.db")
session = atoti.create_session(config=config)

#### Downloading the notebook demo data

In [3]:
# Checking if the file has already been downloaded 
def download_source(cwd: str, url: str, filename: str):
    """This function downloads data file from the given url to the working directory (cwd).

    Args:
        cwd: current working directory.
        url: url for the source file
        filename: filename of the downloaded file
    """
    if os.path.isfile(cwd + "/" + filename):
        print(filename + ' already downloaded.')
    else:
        wget.download(url, out=cwd)

In [4]:
cwd = os.getcwd()
download_source(cwd, 'http://data.atoti.io/notebooks/var/instruments.csv', 'instruments.csv')
download_source(cwd, 'http://data.atoti.io/notebooks/var/instruments_pricing_vol_depth_272.csv', 'instruments_pricing_vol_depth_272.csv')
download_source(cwd, 'http://data.atoti.io/notebooks/var/positions.csv', 'positions.csv')
download_source(cwd, 'http://data.atoti.io/notebooks/var/trading_desk.csv', 'trading_desk.csv')
download_source(cwd, 'http://data.atoti.io/notebooks/var/instruments_pricing_vol_depth_150.csv', 'instruments_pricing_vol_depth_150.csv')

instruments.csv already downloaded.
instruments_pricing_vol_depth_272.csv already downloaded.
positions.csv already downloaded.
trading_desk.csv already downloaded.
instruments_pricing_vol_depth_150.csv already downloaded.


Instruments are financial products. In this notebook they are foreign exchange options.

In [5]:
instruments = session.read_csv('instruments.csv', keys=['instrument_code'], store_name="Instruments")

In [6]:
instruments.head()

,underlying_code,strike,option_type,maturity,contract_size
instrument_code,,,,,
EURUSD=X c 1 @1.251000,EURUSD=X,1.251000,c,1,100000
EURUSD=X p 1 @1.251000,EURUSD=X,1.251000,p,1,100000
EURUSD=X c 1 @1.106200,EURUSD=X,1.106200,c,1,100000
EURUSD=X p 1 @1.106200,EURUSD=X,1.106200,p,1,100000
EURUSD=X c 1 @1.163233,EURUSD=X,1.163233,c,1,100000


The analytics store gives more information on each instrument, more notably:
- The PnL of the previous day
- A vector of the PnLs of the instrument for the last 272 days. PnLs are typically calculated by complex price engines and such vectors would be their result.

In [7]:
analytics = session.read_csv('instruments_pricing_vol_depth_272.csv', keys=['instrument_code'], store_name="Instruments Analytics", sep="|", array_sep=";")

In [8]:
analytics.head()

,pnl_vector,vector_size,pnl,value_change_get_values_last
instrument_code,,,,
EURAUD=X c 1 @1.480500,"doubleVector[272]{12624.202770457445, ...}",272.0,12624.202770,-219.999998
EURAUD=X c 1 @1.578882,"doubleVector[272]{4308.278694271345, ...}",272.0,4308.278694,-219.597787
EURAUD=X c 1 @1.662000,"doubleVector[272]{-1744.4302935895314, ...}",272.0,-1744.430294,-168.476599
EURAUD=X p 1 @1.480500,"doubleVector[272]{-20519.06471441983, ...}",272.0,-20519.064714,0.000002
EURAUD=X p 1 @1.578882,"doubleVector[272]{-10239.767451773394, ...}",272.0,-10239.767452,0.402213


In [9]:
positions_store_types = {
    "quantity": atoti.types.DOUBLE,
    "purchase_price": atoti.types.DOUBLE,
}

Positions give us how many of each instrument we currently hold in our portfolio.  
They are grouped into books.

In [10]:
positions = session.read_csv('positions.csv', keys=['instrument_code','book_id'], store_name="Positions", types=positions_store_types)

In [11]:
positions.head()

underlying  quantity  purchase_price
instrument_code        book_id                                     
EURUSD=X c 1 @1.251000 1         EURUSD=X       1.0             0.0
                       2         EURUSD=X       1.0             0.0
                       3         EURUSD=X       1.0             0.0
                       4         EURUSD=X       1.0             0.0
                       5         EURUSD=X       1.0             0.0

### Data model and cube
We will first join the three previous stores altogether.

In [12]:
positions.join(instruments)
instruments.join(analytics)

Then create an OLAP cube on top of that to start our analysis.

In [13]:
cube = session.create_cube(positions, "Positions")

In auto mode, Atoti creates hierarchies for each string columns, sum and average measures for each numerical column.  
This can of course be fine tuned to either switch to full manual mode and create hierarchies/measures yourself, or simply edit what has been created automatically (adding a hierarchy for a numerical column for example).  
Below you can explore which measures/levels/hierarchies have been automatically created in our cube.

In [14]:
m = cube.measures
h = cube.hierarchies
l = cube.levels
cube

A simple command lets you run the Atoti UI directly in our notebook. Pretty convenient to explore the data you just loaded or make sure the measures defined produce the correct results.

In [16]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

#### Computing the PnL of the previous day

In [16]:
# Let's give this measure a more user friendly name
m["Quantity"] = m["quantity.SUM"]

In [17]:
m["PnL"] = atoti.agg.sum(m["quantity.SUM"] * m["pnl.VALUE"], scope=atoti.scope.origin('instrument_code'))

### Looking a the PnL in various ways
Run the following cells to see the Atoti visualizations

In [22]:
cube.visualize(name="PnL Pivot Table")

Install the Atoti JupyterLab extension to see this widget.

In [26]:
cube.visualize("PnL by Type Chart")

Install the Atoti JupyterLab extension to see this widget.

### Collaboration tools
All the tables/charts created in the notebook can be published and made available in Atoti UI, a user friendly interface where anobody can create dashboards, share them, and dig down in the data.

Atoti UI can be reached with a link using command `session.url`  
Run the cell below to have a look at a dashboard we have prepared using the above chart and pivot table.

In [ ]:
from IPython.display import Markdown as md
md("[" + session.url + "/#/dashboard/ee8](" + session.url + "/#/dashboard/ee8)")

### Customizing hierarchies

In large organizations, books usually belong to business units that inside of them might have smaller sub-business units and different trading desks.  
Atoti lets you add new hierarchies on the fly without having to add columns into existing tables or re-launch time consuming batch computations.

In this example we will import from a file a Business Units > Sub-Business Units > Trading Desks > Book hierarchie defined in a file, and since we already have book IDs linked to our instruments, we will simply use this new information to create an additional hierarchy.

In [27]:
trading_desks = session.read_csv('trading_desk.csv', keys=['book_id'], store_name="Trading Desk")
positions.join(trading_desks)

h["Trading Book Hierarchy"] = {
    'Business Unit':l['business_unit'],
    'Sub Business Unit':l['sub_business_unit'],
    'Trading Desk' : l['trading_desk'], 
    'Book':l['book']
}

The cube structure has been modified on the fly, we can now use the new hierarchy on any visualization.

In [29]:
cube.visualize("Business Hierarchy Pivot Table")

Install the Atoti JupyterLab extension to see this widget.

### Value at Risk

We have vectors of the PnLs of every instrument for the last 272 days.  
First thing we will do is defined a "scaled vector" measure that will multiply those PnLs vectors by the quantities we have of the instruments, aggregate it as a sum above.

In [30]:
scaled_pnl_vector = m["Quantity"] * m["pnl_vector.VALUE"]
m['Position Vector'] = atoti.agg.sum(scaled_pnl_vector, scope=atoti.scope.origin('instrument_code'))

Then, given a confidence level `x`, the value at risk is the `x`th percentile of the scaled vector.  
In more understandable terms, if the confidence level is 95%, based on past PnL values, the value at risk gives a PnL loss that we only had 5% chances to exceed in the past.

In [31]:
m["Confidence Level"] = 0.95
m["VaR"] = atoti.array.percentile(m["Position Vector"], m["Confidence Level"])

In [33]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

95% is an arbitrary value, what if the extreme cases are ten times worse than what we have? Or what if assuming a little more risk would tremendously reduce the Var?

This kind of simulation is pretty easy to put in place with Atoti.  
Below we setup a simulation on measure `Confidence level` then define how it should be calculated in various scenarios (here we only take fixed values).

In [34]:
confidence_levels = cube.setup_simulation("Confidence Level", replace=[m["Confidence Level"]]).scenarios;

confidence_levels["90%"] = 0.90
confidence_levels["98%"] = 0.98
confidence_levels["99%"] = 0.99
confidence_levels["Worst"] = 1.0

Once the simulation is setup, we can access its different values using a new `Confidence level` hierarchy that has automatically been created

In [36]:
cube.visualize("VAR 98% on SBU")

Install the Atoti JupyterLab extension to see this widget.

### Marginal VaR

Let's imagine now that the VaR of a book/desk is too high. Since the financial institution risks losing too much money, it needs to close a few positions in order to reduce the VaR to the maximum.

Since the VaR is a percentile, its aggregation is non-linear, meaning that the sum of the VaRs of all the books in a desk is not equal to the VaR of the desk. So we cannot really look the the books VaRs and close the one with the biggest VaR.

What we will do instead is compute the marginal VaR of each element in the trading book hierarchy, which is the VaR of the parent of an element, but excluding data from this element in the computation.  
Continuing with the example of desk/books, the marginal VaR of a book is the VaR of its desk, but excluding all positions from this book.

Cells below detail this computation.

In [37]:
m["Parent Position Vector Ex"] = atoti.agg.sum(
    m["Position Vector"],
    scope=atoti.scope.siblings(h["Trading Book Hierarchy"], exclude_self=True),
)

In [38]:
m["Parent VaR Ex"] = atoti.array.percentile(m["Parent Position Vector Ex"], m["Confidence Level"])
m["Parent VaR"] = atoti.parent_value(m["VaR"], h["Trading Book Hierarchy"])
m["Marginal VaR"] = m["Parent VaR"] - m["Parent VaR Ex"]

That's it, our marginal VaR is computed, let's have a look at where we could reduce the VaR the most now.

In [40]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

## PnL Models Comparison

The VaR calculation is heavily based on the PnL vectors that depend on the results of our instruments pricers, and the history that we have.  
What would happen if pricers used a different model, or if we changed the amount of history we use to compute the VaR.

Atoti also lets you perform easy simulations on data tables that were loaded.  
Here we have another analytics file with PnL vectors but only for the last 150 days.  
What we will do is ask Atoti to load this new file in the analytics store, but in a new scenario called "Model short volatility".

In [19]:
new_analytics_file = 'instruments_pricing_vol_depth_150.csv'
analytics.scenarios['Model short Volatility'].load_csv(new_analytics_file, sep="|", array_sep=";")

And that's it, no need to re-load any other file, re-define measures or perform batch computations. Everything we have previously defined is available on both our previous and this new scenario.  
Let's have a look at it.

In [21]:
cube.visualize("PnL comparaison Model Short Volatility")

Install the Atoti JupyterLab extension to see this widget.

In [25]:
cube.visualize("VAR Comaraison Model Short Volatility")

Install the Atoti JupyterLab extension to see this widget.